In [1]:
import chromadb
from chromadb.config import Settings
import pickle
import numpy as np
from tqdm import tqdm

In [2]:
with open('../data/doc_to_idx.pkl', 'rb') as f:
    passages = pickle.load(f)

In [3]:
print(type(passages))

<class 'dict'>


In [4]:
print("Top-level keys (sample):", list(passages.keys())[:1])


Top-level keys (sample): ['Property Taxes & Public Education Funding: Inequality in the System Joshua R. Berry April 29, 2011 Final Project Essay – Final Draft URP 6542 – Urban Land Economics The Complex Issues of Public School Funding From its inception, the United States and its citizenry have placed tremendous importance on the education of its youth.']


In [5]:
embeddings = np.load('../data/document_embeddings.npy', allow_pickle=True)

In [6]:
print(type(embeddings))
print("Shape:", embeddings.shape)

<class 'numpy.ndarray'>
Shape: (93270, 256)


In [7]:
# Setup ChromaDB
# Create a persistent ChromaDB client
client = chromadb.PersistentClient(path="./chroma_store")  # Your desired folder

# Check if 'docs' exists before deleting
existing_collections = [col.name for col in client.list_collections()]
if "docs" in existing_collections:
    client.delete_collection("docs")
    
# Create or load a collection
collection = client.get_or_create_collection("docs")


In [ ]:
for i, (passage, embedding) in tqdm(enumerate(zip(passages, embeddings)), total=len(passages)):
    #print(f"Index {i}: {passage} -> {embedding}")

    collection.add(
        ids=[f"doc_{i}"],  # must be a list of strings
        documents=[passage],  # list of 1 document
        embeddings=[embedding.tolist()]  # list of 1 embedding
    )
    
    #Optional: pause to avoid flooding Jupyter
    # if i >= 2000:  #during dev
    #     break


In [9]:
count = collection.count()
print(f"Number of entries in the collection: {count}")

Number of entries in the collection: 93270


In [10]:
# results = collection.get(include=['documents','embeddings'])

# for i in range(min(5, len(results['ids']))):
#     print(f"{i}: ID = {results['ids'][i]}")
#     print(f"   Document: {results['documents'][i]}")
#     # Optional: show part of the embedding
#     print(f"   Embedding (first 5 dims): {results['embeddings'][i][:5]}")



0: ID = doc_0
   Document: Property Taxes & Public Education Funding: Inequality in the System Joshua R. Berry April 29, 2011 Final Project Essay – Final Draft URP 6542 – Urban Land Economics The Complex Issues of Public School Funding From its inception, the United States and its citizenry have placed tremendous importance on the education of its youth.
   Embedding (first 5 dims): [ 0.03073787  0.00905132  0.12641057 -0.03313518  0.03842032]
1: ID = doc_1
   Document: Dry rot is the term given to brown rot decay caused by certain fungi that deteriorate timber in buildings and other wooden construction without an apparent source of moisture. The term is a misnomer because all wood decaying fungi need a minimum amount of moisture before decay begins.
   Embedding (first 5 dims): [ 0.01729996  0.0087871   0.12702529 -0.01890628  0.04625144]
2: ID = doc_2
   Document: How much does ABS Light is on Inspection cost? It depends on the type of car you drive and the auto repair shop you go to